In [130]:
#import the neccesary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import xgboost
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
import pickle

In [131]:
data=pd.read_csv('data_model.csv')
data=data.drop(columns='Unnamed: 0')
data.head()

,meter,meter_reading,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use,hour,time_of_day,month
0,0.0,2.325,25.0,6.0,20.0,0.0,1019.7,0.0,0.0,Office,0,Night,1
1,0.0,2.225,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5,Office,1,Night,1
2,0.0,2.275,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,Office,2,Night,1
3,0.0,2.575,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,Office,3,Night,1
4,0.0,3.075,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,Office,4,Night,1


In [132]:
data.shape

(8784, 13)

In [133]:
#split into features and target
X=data.drop(columns='meter_reading')
y=data['meter_reading']

In [134]:
X.head()

,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use,hour,time_of_day,month
0,0.0,25.0,6.0,20.0,0.0,1019.7,0.0,0.0,Office,0,Night,1
1,0.0,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5,Office,1,Night,1
2,0.0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,Office,2,Night,1
3,0.0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,Office,3,Night,1
4,0.0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,Office,4,Night,1


In [135]:
#one hot encoding the features
X=pd.get_dummies(X)

In [136]:
X.head()

,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,month,primary_use_Entertainment/public assembly,primary_use_Lodging/residential,primary_use_Office,time_of_day_Afternoon,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
0,0.0,25.0,6.0,20.0,0.0,1019.7,0.0,0.0,0,1,0,0,1,0,0,0,1
1,0.0,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5,1,1,0,0,1,0,0,0,1
2,0.0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,2,1,0,0,1,0,0,0,1
3,0.0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,3,1,0,0,1,0,0,0,1
4,0.0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,4,1,0,0,1,0,0,0,1


In [137]:
#80-20 split for train, test respectively
X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=0)

In [138]:
#linear regression
Lr=LinearRegression()
Lr.fit(X_train,y_train)
Lr_pred_test=Lr.predict(X_test)

In [139]:
#decision tree
Dt=DecisionTreeRegressor(max_depth=3,min_samples_split=7,random_state=0)
Dt.fit(X_train,y_train)
Dt_pred_test=Dt.predict(X_test)

In [140]:
#Xgboost
xg=XGBRegressor(max_depth=5,n_estimators=70)
xg.fit(X_train,y_train)
xg_pred_test=xg.predict(X_test)

[14:08:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [141]:
#random forest
Rf=RandomForestRegressor(max_depth=7,n_estimators=100,random_state=0)
Rf.fit(X_train,y_train)
Rf_pred_test=Rf.predict(X_test)

In [142]:
#r2 score of the four models
print('The r2 score for the linear regression model  is ',r2_score(y_test, Lr_pred_test))
print('The r2 score for the Decision tree model  is ',r2_score(y_test, Dt_pred_test))
print('The r2 score for the XGboost model  is ',r2_score(y_test, xg_pred_test))
print('The r2 score for the Random forest model  is ',r2_score(y_test, Rf_pred_test))

The r2 score for the linear regression model  is  0.743349298925913
The r2 score for the Decision tree model  is  0.8046698452256102
The r2 score for the XGboost model  is  0.9292162345288391
The r2 score for the Random forest model  is  0.7850608881499156


In [143]:
#MAE of the four models
print('The mean squared error for the linear regression model  is ',np.sqrt(mean_squared_error(y_test, Lr_pred_test)))
print('The mean squared error for the Decision tree model  is ',np.sqrt(mean_squared_error(y_test, Dt_pred_test)))
print('The mean squared error for the XGboost model  is ',np.sqrt(mean_squared_error(y_test, xg_pred_test)))
print('The mean squared error for the Random forest model  is ',np.sqrt(mean_squared_error(y_test, Rf_pred_test)))

The mean squared error for the linear regression model  is  5.830475953914596
The mean squared error for the Decision tree model  is  5.086481019024826
The mean squared error for the XGboost model  is  3.0619606988757986
The mean squared error for the Random forest model  is  5.335688976341888


#### The xgboost algorithm performs better than the others after hyperparameter tuning, it has the lowest root MAE value and higher r2score. That is the algorithm we are going to choose

In [144]:
#save the file 
pickle.dump(xg, open('Energy.pkl','wb'))

### the model is now ready for deployment!